In [ ]:
import pickle as p
import numpy as np
from plotly import graph_objects as go
import os
from sklearn.mixture import GaussianMixture
from tqdm.notebook import tqdm  # last tqdm update
from scipy.stats import shapiro
from statsmodels.sandbox.stats.multicomp import multipletests 

In [ ]:
first_ten = []
for file in os.listdir('./Adam'):
    with open('./Adam/' + file, 'rb') as inp:
        try:
            first_ten.append(p.load(inp)[0][:10])
        except:
            pass
first_ten = np.array(first_ten)
first_ten.shape

# Распределение градиентов

## Данные

Для первых ста итераций обучения были получены значения градиентов для весов между десятью нейронами на последнем слое и
Предпоследним скрытым слоем с 512 нейронами.

## Первый взгляд

Давайте посмотрим на все градиенты для каждого нейрона за все итерации. Такая апроксимация не слишком математична, так как для каждого веса распределение свое, однако такая интерперетация более наглядна. Полученная величина -- смесь нормальных распределений. Оценим эту смесь через `GaussianMixture` и построим гистограммы.

In [ ]:
from plotly.subplots import make_subplots



data = []
all_grad = []
mixt_est = []


for param in range(10):
    param_grad = first_ten[:, param, :].reshape(-1)
    all_grad.append(param_grad)



models = []

for it in range(10):
    data.append(go.Histogram({
        'x': all_grad[it],
        'opacity': 0.7,
        'histnorm': 'probability density',
        'name': 'param: {} all weights'.format(it+1)
    }))
    cur_model = GaussianMixture(5)
    cur_model.fit(all_grad[it].reshape(-1, 1))
    x = np.linspace(np.min(all_grad[it]), np.max(all_grad[it]), 200).reshape(-1, 1)
    y = cur_model.score_samples(x).reshape(-1)
    if it == 0:
        mixt_est.append(go.Scatter({
            'x': x.reshape(-1),
            'y': np.exp(y),
            'marker': {
                'color': 'red'
            },
        
            'name': 'Gaussian mixture estimator'
        }))
        continue
        
    mixt_est.append(go.Scatter({
        'x': x.reshape(-1),
        'y': np.exp(y),
        'marker': {
            'color': 'red'
        },
        
        'showlegend': False
    }))
    

fig = make_subplots(cols=4, rows=3)

for idx, trace in enumerate(data):
    fig.add_trace(mixt_est[idx], col=idx//3 + 1, row=idx%3+1)
    fig.add_trace(trace, col=idx//3 + 1, row=idx%3+1)
    
    
layout = go.Layout({
    'title': 'Distribution of gradients',
    'width': 1000,
    'height': 700,
    'template': 'plotly_white',
    #'legend_orientation': 'h',
    'legend': {
        'x': 0.8,
        'y': 0.4
    }
})
fig.update_layout(layout)
fig.show()

## Вывод

Как видим данные вполне неплохо подходят под данное распределение, а значит мы можем продолжить нашу проверку на нормальность градиентов каждого веса.

## Критерий согласия Шапиро-Уилка

Для каждого веса для каждого нейрона проверим гипотезу $H_0: \text{"Градиенты на данном весе распределены нормально"} $ против альтернативы $H_1: \text{"Иначе"} $. Для этого воспользуемся критерием согласия Шапиро-Уилка.

In [ ]:
alpha = 0.05

p_values = []
for param in tqdm(range(10)):
    p_values_param = []
    for weight in range(512):
        p_value = shapiro(first_ten[:, param, weight])[1]
        p_values_param.append(p_value)
    p_values.append(p_values_param)

p_values = np.array(p_values)

In [ ]:
p_values.shape

Итак, число не отвергнутых гипотез и отвергнутых гипотез можем видеть в выводе следующе ячейки.

In [ ]:
accepted = p_values[p_values > alpha]
rejected = p_values[p_values <= alpha]
len(accepted), len(rejected)

Так же посмотрим на каждый нейрон и веса, принадлежащие конкретному нейрону.

In [ ]:
for idx, param_p in enumerate(p_values):
    accepted = len(param_p[param_p > alpha])
    rejected = len(param_p[param_p <= alpha])
    print('For parameter {} Shapiro-Wilk test results are:\n accepted {}; rejected {}; percentage accepted {:.2f}'.format(
    idx+1, accepted, rejected, accepted/len(param_p)*100))

## Множественная проверка гипотез

Воспользуемся поправками Холма и Бенджамини-Хохберга. Посмотрим какая из них даст более мощный результат и воспользуемся такой.

Будем проверять гипотезу $H_0: \text{"В целом градиенты распределены нормально"} $ против $ H_1: \text{"Иначе"} $

### Метод Холма

In [ ]:
reject, p_corrected, a1, a2 = multipletests(p_values.reshape(-1), 
                                            alpha = 0.05, 
                                            method = 'holm')

In [ ]:
reject.sum()

### Метод Бенджамини-Хохберга

In [ ]:
reject, p_corrected, a1, a2 = multipletests(p_values.reshape(-1), 
                                            alpha = 0.05, 
                                            method = 'fdr_bh')

In [ ]:
reject.sum()

### Вывод 
С помощью Метод Бенджамини-Хохберга удалось отвергнуть больше гипотез, а значит он является более мощным в данном случае.

# Подсчет дисперсии learning rate

Рассмотрим градиент, для которого гипотеза о нормальности не отвергается:

In [ ]:
norm_gradients = first_ten[:, ~reject.reshape(10, 512)]
norm_gradients.shape

In [ ]:
# np.random.seed(42)
grad_number = np.random.randint(1, norm_gradients.shape[1])

In [ ]:
grad_number

grad_number = 309, 774 — большие отличия

In [ ]:
import seaborn as sns
%matplotlib inline

sns.set(style='whitegrid', font_scale=1.3, palette='Set3')
grad_sample = norm_gradients[:, grad_number]

sns.distplot(grad_sample)

In [ ]:
def adaptive_learning_rate(gradients, beta=0.999):
    coefs = beta ** np.arange(len(gradients), 0, -1)
    sum_ = np.sum(coefs*gradients**2)
    coef = np.sqrt( (1 - beta**len(gradients)) / ( (1 - beta) * sum_ ) )
    return coef


def rho_t(t, beta=0.999):
    rho_inf = 2 / (1 - beta) - 1
    rho = rho_inf - 2 * t * (beta**t) / (1 - beta**t)
    return rho


def var_psi(sigma, t):
    return rho_t(t) / (2 * (rho_t(t) - 2) * (rho_t(t) - 4) * sigma**2)

from scipy.optimize import minimize

In [ ]:
def psi_variance_plot(rejected=True):
    if rejected:
        norm_gradients = first_ten[:, reject.reshape(10, 512)]
    else:
        norm_gradients = first_ten[:, ~reject.reshape(10, 512)]
        
    grad_number = np.random.randint(1, norm_gradients.shape[1])
    grad_sample = norm_gradients[:, grad_number]
    
    from sklearn.utils import resample
    N_samples = 1000
    alpha = 0.95

    psi_var = []
    for i in tqdm(range(1, len(grad_sample) + 1)):
        grad_slice = grad_sample[:i]
        bootstrap_samples = np.array([resample(grad_slice) 
                                      for i in range(N_samples)])
        psi_bootstrap = [adaptive_learning_rate(cur_sample) 
                         for cur_sample in bootstrap_samples]
        psi_var.append(np.var(psi_bootstrap))
    
    x = np.linspace(0, 100, 101)
    real = go.Scatter({
        'x': x[5:],
        'y': psi_var[5:],
        'name': 'Real variance'
    })

    theor = go.Scatter({
        'x': x[5:],
        'y': var_psi(np.std(grad_sample), x[5:]),
        'name': 'Theoretical variance'
    })


    return [real, theor]

Для начала сравним реальную и теоретическую дисперсиию $\psi(g_1, \dots, g_t)$ в зависимости от $t$ для случая, когда гипотеза о нормальности отверглась

In [ ]:
layout = go.Layout({
    'title': 'Estimated variance',
    'xaxis': {
        'title': 'iteration'
    },
    'yaxis': {
        'title': 'variance'
    },
    'template': 'plotly_white',
    'width': 700,
    'height': 500,
})
go.Figure(data=psi_variance_plot(), layout=layout)

Построим такой же график для случая, когда гипотеза о нормальности не отвергается:

In [ ]:
go.Figure(data=psi_variance_plot(rejected=False), layout=layout)

Можно заметить, что на первых итерациях дисперсия градиентов, распределение которых отлично от нормального, в основном выше.

 # RAdam

In [ ]:
first_ten_radam = []
for file in os.listdir('./RAdam'):
    with open('./RAdam/' + file, 'rb') as inp:
        try:
            first_ten_radam.append(p.load(inp)[0][:10])
        except:
            pass
first_ten_radam = np.array(first_ten_radam)
first_ten_radam.shape

Проверка гипотез методом Холма:

In [ ]:
alpha = 0.05

p_values_radam = []
for param in tqdm(range(10)):
    p_values_param = []
    for weight in range(512):
        p_value = shapiro(first_ten[:, param, weight])[1]
        p_values_param.append(p_value)
    p_values_radam.append(p_values_param)

p_values_radam = np.array(p_values_radam)

In [ ]:
reject_radam, p_corrected, a1, a2 = multipletests(p_values.reshape(-1), 
                                            alpha = 0.05, 
                                            method = 'holm')

## Оценка дисперсии для RAdam

In [ ]:
def rectified_psi(gradients, beta=0.999):
    t = len(gradients)
    rho_inf = 2 / (1 - beta) - 1
    rho = rho_t(t)
    r_t = np.sqrt((rho - 4) * (rho - 2) * rho_inf / ((rho_inf - 4) * (rho_inf - 2) * rho))
    return r_t * adaptive_learning_rate(gradients, beta)

In [ ]:
def rectified_psi_variance_plot(rejected=True):
    if rejected:
        norm_gradients = first_ten_radam[:, reject_radam.reshape(10, 512)]
    else:
        norm_gradients = first_ten_radam[:, ~reject_radam.reshape(10, 512)]
        
    grad_number = np.random.randint(1, norm_gradients.shape[1])
    grad_sample = norm_gradients[:, grad_number]
    
    from sklearn.utils import resample
    N_samples = 1000
    alpha = 0.95

    psi_var = []
    for i in tqdm(range(1, len(grad_sample) + 1)):
        grad_slice = grad_sample[:i]
        bootstrap_samples = np.array([resample(grad_slice) 
                                      for i in range(N_samples)])
        psi_bootstrap = [rectified_psi(cur_sample) 
                         for cur_sample in bootstrap_samples]
        psi_var.append(np.var(psi_bootstrap))
    
    x = np.linspace(0, 100, 101)
    real = go.Scatter({
        'x': x[5:],
        'y': psi_var[5:],
        'name': 'Real variance'
    })
    
    return real

Построим аналогичный график для `rectified_psi`:

In [ ]:
go.Figure(data=[rectified_psi_variance_plot()], layout=layout)

In [ ]:
go.Figure(data=[rectified_psi_variance_plot(rejected=False)], layout=layout)

Как мы можем заметить, RAdam понижает дисперсию learning rate для нормальных и отличных от нормального весов примерно одинаково.

 # AdamWarmup

In [ ]:
first_ten_w = []
for file in os.listdir('./AdamW'):
    with open('./AdamW/' + file, 'rb') as inp:
        try:
            first_ten_w.append(p.load(inp)[0][:10])
        except:
            pass
first_ten_w = np.array(first_ten_w)
first_ten_w.shape

In [ ]:
alpha = 0.05

p_values_w = []
for param in tqdm(range(10)):
    p_values_param = []
    for weight in range(512):
        p_value = shapiro(first_ten[:, param, weight])[1]
        p_values_param.append(p_value)
    p_values_w.append(p_values_param)

p_values_w = np.array(p_values_w)